本文学习于https://tensorflow.google.cn/guide/datasets#processing_multiple_epochs 

# 导入数据

借助 `tf.data` API，您可以根据简单的可重用片段构建复杂的输入管道。例如，**图片模型**的管道可能会汇聚分布式文件系统中的文件中的数据、对每个图片应用随机扰动，并将随机选择的图片合并成用于训练的批次。**文本模型**的管道可能包括从原始文本数据中提取符号、根据对照表将其转换为嵌入标识符，以及将不同长度的序列组合成批次数据。使用` tf.data `API 可以轻松处理大量数据、不同的数据格式以及复杂的转换。

`tf.data` API 在 `TensorFlow `中引入了两个新的抽象类：
1. `tf.data.Dataset` 表示一系列元素，其中每个元素包含一个或多个` Tensor `对象。例如，在图像管道中，元素可能是单个训练样本，具有一对表示图像数据和标签的张量。可以**通过两种不同的方式来创建数据集**：
    1. 创建来源（例如 `Dataset.from_tensor_slices()`），以通过一个或多个` tf.Tensor `对象构建数据集
    2. 应用转换（例如` Dataset.batch()`），以通过一个或多个` tf.data.Dataset `对象构建数据集
    
2. `tf.data.Iterator `提供了**从数据集中提取元素的主要方法**。`Iterator.get_next() `返回的操作会在执行时生成` Dataset `的下一个元素，并且此操作通常充当输入管道代码和模型之间的接口。最简单的迭代器是“单次迭代器”，它与特定的` Dataset `相关联，并对其进行一次迭代。要实现更复杂的用途，您可以通过` Iterator.initializer `操作使用不同的数据集重新初始化和参数化迭代器，这样一来，您就可以在同一个程序中对训练和验证数据进行多次迭代（举例而言）。

# 基本机制

本指南的这一部分介绍了**创建不同种类的` Dataset `和 `Iterator `对象的基础知识，以及如何从这些对象中提取数据**。

**要启动输入管道，您必须定义来源**。例如，要通过内存中的某些张量构建` Dataset`，您可以使用 `tf.data.Dataset.from_tensors() `或` tf.data.Dataset.from_tensor_slices()`。或者，如果输入数据以推荐的` TFRecord `格式存储在磁盘上，那么您可以构建` tf.data.TFRecordDataset`。

一旦有了` Dataset `对象，可以将其转换为**新的` Dataset`**，方法是链接` tf.data.Dataset `对象上的方法调用。例如，您可以应用单元素转换，例如` Dataset.map()`（为每个元素应用一个函数），也可以应用多元素转换（例如` Dataset.batch()`）。要了解转换的完整列表，请参阅` tf.data.Dataset `的文档。

**消耗` Dataset `中值的最常见方法是构建迭代器对象**。通过此对象，可以一次访问数据集中的一个元素（例如通过调用 `Dataset.make_one_shot_iterator()`）。`tf.data.Iterator` 提供了两个操作：`Iterator.initializer`，您可以通过此操作（重新）初始化迭代器的状态；以及` Iterator.get_next()`，此操作返回对应于有符号下一个元素的` tf.Tensor `对象。根据您的使用情形，您可以选择不同类型的迭代器，下文介绍了具体选项。

## 数据集结构

**一个数据集包含多个元素，每个元素的结构都相同**。一个元素包含一个或多个` tf.Tensor `对象，这些对象称为**组件**。每个组件都有一个` tf.DType`，表示张量中元素的类型；以及一个` tf.TensorShape`，表示每个元素（可能部分指定）的静态形状。

您可以通过` Dataset.output_types `和 `Dataset.output_shapes` **属性**检查数据集元素各个**组件**的推理类型和形状。这些属性的嵌套结构映射到元素的结构，此元素可以是单个张量、张量元组，也可以是张量的嵌套元组。例如：

`tf.data.Dataset()`的三个属性:
1. Dataset.output_types: 返回此数据集元素的每个组件的数据类型
2. Dataset.output_shapes: 返回此数据集元素的每个组件的形状
3. Dataset.output_classes: 返回此数据集元素的每个组件的类

### 生成一个组件的数据集：随机生成4 x 10 的数据，并将其包装为数据集

我们调用函数`tf.random_uniform()`来生成4行10列服从均匀分布的随机数据，之后调用函数`tf.data.Dataset.from_tensor_slices()`将其包装为数据集`dataset1`。数据集`dataset1`包含4个元素，每个元素又包含10个元素，在这里这10个元素便是一个组件，因为它们是由一个函数统一生成的。

In [1]:
import tensorflow as tf

# 将张量tf.random_uniform([4,10])创建为数据集
sess = tf.Session()
print(sess.run(tf.random_uniform([4,10])))
print()
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4,10]))
print('dataset1.output_types: ',dataset1.output_types)
print('dataset1.output_shapes: ',dataset1.output_shapes)

[[0.2741233  0.984205   0.30764318 0.7318605  0.700199   0.5479107
  0.39938617 0.66468716 0.5732206  0.09478366]
 [0.385113   0.66434634 0.00105882 0.4056605  0.1529671  0.47529948
  0.39893448 0.9187614  0.3400545  0.23579264]
 [0.2882105  0.49117863 0.7593796  0.2701422  0.4975921  0.03151095
  0.6039736  0.38248336 0.00157523 0.5611272 ]
 [0.23421323 0.9629564  0.9917984  0.00435078 0.14973891 0.61171854
  0.21859157 0.10950458 0.03355992 0.46188796]]

dataset1.output_types:  <dtype: 'float32'>
dataset1.output_shapes:  (10,)


### 生成含有两个组件的数据集

这里将一个数组与一个矩阵合并成一个数据集：

1. 数组由函数`tf.random_uniform([4])`生成，是只有4个元素的数组，这是一个组件
2. 矩阵由函数`tf.random_uniform([4,10])`生成，是一个4行10列的矩阵，这是一个组件
3. 我们的目标是将这两个组件合并到一个数据集中，即数组的每个元素与矩阵每一行的元素拼接在一起
4. 这个操作可用于将训练数据和标签拼接在一起

In [2]:
print(sess.run([tf.random_uniform([4]),tf.random_uniform([4,10],maxval=100,dtype=tf.int32)]))
print()
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]),tf.random_uniform([4,10],maxval=100,dtype=tf.int32))
)
print('dataset2.output_types: ',dataset2.output_types)
print('dataset2.output_shapes: ',dataset2.output_shapes)

[array([0.21377683, 0.74616075, 0.31666708, 0.03640652], dtype=float32), array([[68, 70, 76, 36, 48, 13, 65, 55, 64,  6],
       [85, 80, 12, 59, 42, 40, 26, 51, 15, 68],
       [58, 56, 51, 91, 24, 60, 31, 75, 86, 76],
       [88, 98, 98, 95, 87, 21, 91, 66, 31, 40]])]

dataset2.output_types:  (tf.float32, tf.int32)
dataset2.output_shapes:  (TensorShape([]), TensorShape([Dimension(10)]))


### 将两个数据集拼接为一个大数据集

这里使用函数`tf.data.Dataset.zip()`来完成两个数据集的拼接。

In [3]:
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))
print('dataset3.output_types: ',dataset3.output_types)
print('dataset3.output_shapes: ',dataset3.output_shapes)

dataset3.output_types:  (tf.float32, (tf.float32, tf.int32))
dataset3.output_shapes:  (TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(10)])))


### 为元素的每个组件命名

**为元素的每个组件命名**通常会带来便利性，例如，如果它们表示训练样本的不同特征。除了元组之外，还可以使用 `collections.namedtuple `或将字符串映射到张量的字典来表示` Dataset `的单个元素。

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(
        {
            "a":tf.random_uniform([4]), # 为组件命名为"a"
            "b":tf.random_uniform([4,10],maxval=100,dtype=tf.int32) # 为组件命名为"b"
        }
)
print('dataset.output_types: ',dataset.output_types)
print('dataset.output_shapes: ',dataset.output_shapes)

dataset.output_types:  {'a': tf.float32, 'b': tf.int32}
dataset.output_shapes:  {'a': TensorShape([]), 'b': TensorShape([Dimension(10)])}


### 对数据集中的每个元素做变换

`Dataset` 转换支持任何结构的数据集。在使用 `Dataset.map(func)`、`Dataset.flat_map(func) `和 `Dataset.filter(func)`转换时（这些转换会对每个元素应用一个函数func），**元素结构决定了函数的参数，即元素的组件个数，每个组件对应一个参数**。**注意这个`func`的构成，只能调用TensorFlow的指令来实现，或者调用`tf.py_func()` 来应用任意` Python `逻辑。**

#### Dataset.map(func)：转换数据

![](./image/map.jpg)

一般我们直接用`lambda`来构造一个转换函数`func`。如果我们要使用比较复杂的函数`func`，可以在外部定义该函数，然后将该函数传递给`map()`函数即可。下面我们对数据集`dataset1`中的每个元素都加上`20`，结果如下：

<font color='red'>对含有一个组件的数据集</font>

In [5]:
# 转换数据集
dataset_1 = dataset1.map(lambda x:x+20.0)
# 初始化数据集
iterator = dataset_1.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))

Instructions for updating:
Colocations handled automatically by placer.
[20.82788  20.11873  20.71489  20.521261 20.436136 20.14803  20.955408
 20.649748 20.41334  20.862717]
[20.149113 20.422483 20.037062 20.979362 20.842241 20.197247 20.682177
 20.89204  20.165327 20.39434 ]
[20.01961  20.013868 20.328447 20.649069 20.042847 20.586498 20.01889
 20.09459  20.061747 20.056982]
[20.569563 20.798359 20.531923 20.381258 20.097027 20.01247  20.918818
 20.030113 20.248314 20.019457]


<font color='red'>对含有两个组件的数据集</font>

In [6]:
dataset_2 = dataset2.map(lambda x,y: (x + 20.0,y+20))
# 初始化数据集
iterator = dataset_2.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))

(20.574408, array([ 98,  56,  70,  41, 113, 102,  60, 110,  85, 102]))
(20.275204, array([115,  66, 110,  51, 101,  65, 111, 113,  39,  65]))
(20.37418, array([ 69,  51,  52,  24,  70,  28,  88, 104,  69,  66]))
(20.760666, array([63, 25, 30, 49, 68, 85, 46, 75, 75, 35]))


In [7]:
import tensorflow as tf
import numpy as np

def func(x):
    return np.sinh(x)

data = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4,10]))
data = data.map(lambda x:tf.py_func(func,[x],tf.float32))
# 初始化数据集
iterator = data.make_initializable_iterator()
# 开启会话执行初始化
sess = tf.Session()
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
[0.03983311 0.44234782 0.01308216 0.2136305  0.85463643 0.91511595
 0.9265351  0.23242007 0.26543185 0.5131738 ]
[1.0526003  0.23431282 1.0509871  0.8666484  0.49518317 0.29676712
 0.40720302 1.0404855  0.49858573 0.15077697]
[0.392435   0.6414319  0.7045138  0.39710397 0.5093797  0.53187764
 0.57883656 0.47780544 0.01682623 0.5834313 ]
[0.17444782 0.9128559  0.5227732  1.1227753  1.0389084  0.50549686
 1.0406598  0.22438155 0.61294913 0.45159495]


#### Dataset.flat_map(func)：转换+平整

功能与`Dataset.map(func)`一样，只是将数据集平整输出了，即每条数据中的元素按照它们在数据集中的顺序，降成一维输出。注意这里`func`返回的必须是`tf.data.Dataset`类型数据，可以使用函数`tf.data.Dataset.from_tensors()`来包裹结果，以生成`tf.data.Dataset`类型数据。

<font color='red'>对含有一个组件的数据集应用</font>

In [8]:
# 转换数据集
dataset_1 = dataset1.map(lambda x:x+20.0)
# 初始化数据集
iterator = dataset_1.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))

print()
print('- - - - - - - -  - - - -- 分割线- - - - - - - - - -- - -- - ')
print()

# dataset1原来的数据形状是{[[1,..],[1,..],[1,..],[1,..]]}，花括号指代dataset1是一个元素
# lambda 函数的作用是,将dataset1切片为[[1,..],[1,..],[1,..],[1,..]]，切分成5个元素，并且每个元素都加20(张量运算)
# flat_map之后数据形状是[1,...]
dataset_1 = dataset1.flat_map(lambda x:tf.data.Dataset.from_tensor_slices(x+20))
# 初始化数据集
iterator = dataset_1.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(40):
    # 输出数据集中的每条数据
    print(sess.run(x))

[20.395962 20.91777  20.56306  20.129417 20.0522   20.0366   20.038157
 20.652414 20.682844 20.224464]
[20.115744 20.771912 20.881048 20.423473 20.644499 20.850859 20.546711
 20.33451  20.257936 20.588638]
[20.021235 20.066236 20.970585 20.01279  20.006866 20.273407 20.411516
 20.423504 20.885063 20.073126]
[20.338827 20.370829 20.70344  20.716917 20.440407 20.457098 20.58464
 20.375034 20.402306 20.575335]

- - - - - - - -  - - - -- 分割线- - - - - - - - - -- - -- - 

20.466047
20.882013
20.470036
20.950432
20.630121
20.469296
20.846058
20.339079
20.29896
20.947474
20.732706
20.494928
20.802444
20.02502
20.854023
20.623781
20.691938
20.947159
20.575966
20.332209
20.19943
20.051043
20.600224
20.031422
20.740742
20.255232
20.286879
20.067488
20.346268
20.31277
20.70106
20.433655
20.232208
20.920704
20.346455
20.40165
20.43862
20.023136
20.128252
20.857567


由于初始化两次`dataset1`数据集，因此输出结果会有不同.

<font color='red'>对含有两个组件的数据集应用</font>

In [9]:
dataset_2 = dataset2.flat_map(lambda x,y:tf.data.Dataset.from_tensors((x+20,y+20)))
# 初始化数据集
iterator = dataset_2.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))
print()
print('- - - - - - - 分割线- - - - -- - -- -- ')
print()

dataset_2 = dataset2.flat_map(lambda x,y:tf.data.Dataset.from_tensors(x))
# 初始化数据集
iterator = dataset_2.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))
print()
print('- - - - - - - 分割线- - - - -- - -- -- ')
print()
dataset_2 = dataset2.flat_map(lambda x,y:tf.data.Dataset.from_tensors(y))
# 初始化数据集
iterator = dataset_2.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))

(20.500332, array([ 94,  45,  60, 100,  41,  92,  21,  56,  82,  49]))
(20.2988, array([ 53,  66,  47,  56,  52,  35,  96, 106,  73,  74]))
(20.372349, array([ 55,  61,  33, 104,  88,  49,  50,  37,  97,  90]))
(20.344847, array([104,  83,  65,  63,  49,  30,  63,  45,  34,  69]))

- - - - - - - 分割线- - - - -- - -- -- 

0.99924123
0.19520259
0.30941665
0.28634322

- - - - - - - 分割线- - - - -- - -- -- 

[33 41 19 97 43 42  8 18 29 32]
[71 22 67 13 24 79 91  2 57 80]
[35  3 90 51  4 87 17 74 57 11]
[26 21 33 62 12 85 84 84 98 10]


#### Dataset.filter(func)：过滤

![](./image/filter.jpg)

`Filters this dataset according to func.`

`func`的作用：将张量的嵌套结构（由`Dataset.output_shapes`和`Dataset.output_types`定义的形状和类型）映射到标量`tf.bool`张量的函数

返回：`The Dataset containing the elements of this dataset for which func is True.`，即返回包含func为真的元素组成的数据集。

In [10]:
data = tf.data.Dataset.from_tensor_slices(tf.range(10))
data = data.filter(lambda x:x>5)
# 初始化数据集
iterator = data.make_initializable_iterator()
# 开启会话执行初始化
sess.run(iterator.initializer)
# 构造迭代器
x = iterator.get_next()
for i in range(4):
    # 输出数据集中的每条数据
    print(sess.run(x))

6
7
8
9


## 创建迭代器

**构建了表示输入数据的 `Dataset `后**，下一步就是**创建迭代器` Iterator `来访问该数据集中的元素**。`tf.data` API 目前支持下列迭代器，复杂程度逐渐增大。

### 单次迭代器

**单次迭代器**是最简单的迭代器形式，**仅支持对数据集进行一次迭代，不需要显式初始化**。单次迭代器可以处理基于队列的现有输入管道支持的几乎所有情况，但它们不支持参数化。以 `Dataset.range() `为例：

In [11]:
import tensorflow as tf

# 创建数据集
dataset = tf.data.Dataset.range(100)
# 创建单次迭代器
iterator = dataset.make_one_shot_iterator()
# 会持续一个一个返回下一个元素，直到数据集耗尽
next_element = iterator.get_next()
for i in range(100):
    value = sess.run(next_element)
    print(value,end=',')
    assert i == value

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,

注意：目前，单次迭代器是唯一易于与 `Estimator `搭配使用的类型。

### 可初始化迭代器

您需要**先运行显式 `iterator.initializer` 操作**，然后才能使用**可初始化迭代器**。虽然有些不便，但它**允许您使用一个或多个` tf.placeholder() `张量**（可在初始化迭代器时馈送）**参数化数据集的定义**。继续以` Dataset.range() `为例：

In [12]:
import tensorflow as tf

max_value = tf.placeholder(tf.int64,shape=[]) # shape=[] 或shape=None，表示该张量是一维的
dataset = tf.data.Dataset.range(max_value)
# 创建可初始化迭代器
iterator = dataset.make_initializable_iterator()
# 不停地获取迭代，以获取下一个元素
next_element = iterator.get_next()

# 初始化迭代器操作
sess.run(iterator.initializer,feed_dict={max_value:10})
for i in range(10):
    value = sess.run(next_element)
    print(value,end=',')
    assert i == value

0,1,2,3,4,5,6,7,8,9,

### 可重新初始化迭代器

**可重新初始化迭代器**可以**通过多个不同的 `Dataset `对象进行初始化**。例如，您可能有一个训练输入管道，它会对输入图片进行随机扰动来改善泛化；还有一个验证输入管道，它会评估对未修改数据的预测。这些管道通常会使用不同的` Dataset `对象，这些对象具有相同的结构（即每个组件具有相同类型和兼容形状）。

In [13]:
# 为训练集和验证集定义相同的结构
training_dataset = tf.data.Dataset.range(100).map(
    lambda x:x+tf.random_uniform([],-10,10,tf.int64) # 表示从-10到10之间生成均匀分布的一个随机值
)
validation_dataset = tf.data.Dataset.range(50)

# 可重新初始化迭代器由它的结构来定义。
# 我们可以使用训练集和验证集的 output_types 和 output_shapes 属性
# 因为它们是兼容的
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                          training_dataset.output_shapes
                                          )
next_element = iterator.get_next()
# 创建可重新初始化迭代器
training_init_op = iterator.make_initializer(training_dataset)
# 创建可重新初始化迭代器
validation_init_op = iterator.make_initializer(validation_dataset)

# 执行20个epoch
for _ in range(20):
    # 初始化训练集迭代器操作
    sess.run(training_init_op)
    for _ in range(100):
        print(sess.run(next_element),end=',')
    print('\n')
    
    # 初始化验证集迭代器操作
    sess.run(validation_init_op)
    for _ in range(50):
        print(sess.run(next_element),end=',')
    print('\n')

9,-8,-2,9,2,14,0,5,7,-1,9,13,4,6,12,7,13,15,13,10,20,22,13,30,18,17,35,28,20,21,30,38,26,31,43,35,37,44,30,37,42,47,35,34,38,52,37,55,39,41,58,43,54,59,55,51,59,54,54,68,57,70,59,53,55,67,71,73,74,67,79,70,67,75,82,72,84,85,87,70,86,73,89,88,79,91,84,88,78,96,96,88,93,85,101,85,96,100,93,101,

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,

0,5,-7,9,3,9,11,8,15,5,9,10,13,18,15,9,17,7,11,10,27,23,12,18,33,33,19,25,21,19,35,33,33,39,24,39,34,36,32,40,43,38,48,50,46,42,40,49,54,51,58,41,55,45,54,50,65,53,51,64,69,69,64,69,57,63,70,74,66,70,65,63,79,78,76,80,77,75,75,70,85,72,85,73,86,85,85,96,89,90,91,91,101,84,90,95,96,92,91,98,

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,

3,3,11,7,0,5,8,7,16,12,0,7,18,20,8,21,11,11,26,25,17,11,29,25,26,26,30,33,28,35,25,21,25,33,43,28,40,38,40,38,34,37,48,34,34,41,

-3,-2,9,6,5,-3,15,12,13,10,11,18,21,10,21,9,19,16,20,11,25,30,21,18,24,25,33,25,21,38,26,31,23,34,42,30,32,31,41,32,47,44,47,43,53,36,44,52,45,54,57,57,57,55,54,64,64,55,59,52,63,69,69,63,67,58,59,66,75,72,78,73,73,68,72,72,85,67,75,75,75,89,73,84,74,90,92,88,84,94,88,84,99,84,96,96,86,93,102,95,

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,



### 可馈送迭代器

**可馈送迭代器**可以与` tf.placeholder` 一起使用，以选择所使用的 `Iterator`（在每次调用 `tf.Session.run` 时）（通过熟悉的 `feed_dict` 机制）。它提供的功能与**可重新初始化迭代器**的相同，**但在迭代器之间切换时不需要从数据集的开头初始化迭代器**。例如，以上面的同一训练和验证数据集为例，您可以使用` tf.data.Iterator.from_string_handle `定义一个可让您在两个数据集之间切换的可馈送迭代器：

In [14]:
import tensorflow as tf

# 定义训练集和验证集以相同的结构
training_dataset = tf.data.Dataset.range(100).map(
  lambda x : x + tf.random_uniform([],-10,10,tf.int64)
).repeat() # 训练集不断重复，因此是无限的
validation_dataset = tf.data.Dataset.range(50)

# 可馈送迭代器由一个占位符及其结构所决定。
# 我们可以使用训练集和验证集的output_types和output_shapes属性
# 因为它们有完全相同的结构
# 创建可馈送迭代器
handle = tf.placeholder(tf.string,shape=[]) # 字符串类型，且维度为一维

iterator = tf.data.Iterator.from_string_handle(
    handle,training_dataset.output_types,training_dataset.output_shapes
)
next_element = iterator.get_next()

# 你可以将可馈送迭代器与各种不同类型的迭代器(例如，单词迭代器和可初始化迭代器)一起使用
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

sess = tf.Session()
# Iterator.string_handle()方法返回一个可以被评估和被用于馈送给handle占位符的张量
training_handle = sess.run(training_iterator.string_handle())
validation_handle = sess.run(validation_iterator.string_handle())

# 循环，在训练集和测试集之间选择
count = 1
while count > 0:
    # 使用训练集运行200步。请注意，训练集是无限的，我们从上一个while循环迭代终止的地方恢复
    # 这里只循环一次即可
    for _ in range(200):
        print(sess.run(next_element,feed_dict={handle:training_handle}),end=',')
    print('\n')
    print()
    # 可初始化迭代器需要显示初始化，而单词迭代器无需
    sess.run(validation_iterator.initializer)
    for _ in range(50):
        print(sess.run(next_element,feed_dict={handle:validation_handle}),end=',')
    print('\n')
    count -= 1

2,-6,-3,-2,-4,9,13,8,-2,13,5,12,15,13,7,17,6,13,13,13,21,17,26,16,26,34,22,17,21,26,25,38,25,25,38,39,36,35,44,38,40,42,49,49,50,42,43,44,54,39,52,58,60,59,53,58,55,57,57,57,69,64,62,57,71,73,65,58,61,63,66,77,64,72,73,75,83,80,76,84,72,90,74,91,84,76,87,95,94,80,94,85,97,94,103,88,94,89,94,99,9,10,2,4,1,2,15,1,15,0,7,20,19,13,4,20,21,8,15,12,22,20,30,22,31,17,16,22,25,28,30,28,34,34,40,38,37,44,43,31,30,44,45,40,39,50,37,47,49,55,54,46,60,43,49,59,56,58,65,50,55,59,69,63,69,58,59,59,70,68,62,68,71,70,78,77,69,70,74,79,83,78,81,80,88,84,95,91,94,93,94,93,89,97,91,100,104,103,97,101,


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,



## 消耗迭代器中的值

`Iterator.get_next() `方法返回一个或多个 `tf.Tensor `对象，这些对象对应于迭代器有符号的下一个元素。每次评估这些张量时，它们都会获取底层数据集中下一个元素的值。

**请注意，与 `TensorFlow `中的其他有状态对象一样，调用 `Iterator.get_next()`并不会立即使迭代器进入下个状态。您必须在` TensorFlow `表达式中使用此函数返回的` tf.Tensor `对象，并将该表达式的结果传递到` tf.Session.run()`，以获取下一个元素并使迭代器进入下个状态**。

如果迭代器到达数据集的末尾，则执行` Iterator.get_next() `操作会产生` tf.errors.OutOfRangeError`。在此之后，迭代器将处于不可用状态；如果需要继续使用，则必须对其重新初始化。

### 简单方法的消耗迭代器中的值

In [15]:
dataset = tf.data.Dataset.range(5) # 创建数据集
iterator = dataset.make_initializable_iterator() # 给这个数据集创建一个迭代器
next_element = iterator.get_next() # 返回数据集的下一条数据

# 添加运算
result = tf.add(next_element,next_element)
sess.run(iterator.initializer)
print(sess.run(result))
print(sess.run(result))
print(sess.run(result))
print(sess.run(result))
print(sess.run(result))

# 如果数据集耗尽，则抛出异常，我们便捕获它
try :
    sess.run(result)
except tf.errors.OutOfRangeError:
    print('End of dataset')

0
2
4
6
8
End of dataset


### 将训练循环封装在try-except中

In [16]:
dataset = tf.data.Dataset.range(5) # 创建数据集
iterator = dataset.make_initializable_iterator() # 给这个数据集创建一个迭代器
next_element = iterator.get_next() # 返回数据集的下一条数据

# 添加运算
result = tf.add(next_element,next_element)
sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(result))
    except tf.errors.OutOfRangeError:
        break

0
2
4
6
8


### 消耗具有嵌套结构的数据集，即含有多个组件

如果数据集的每个元素都具有嵌套结构，则 `Iterator.get_next()` 的返回值将是一个或多个` tf.Tensor `对象，这些对象具有相同的嵌套结构：

In [17]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4,10]))
dataset2 = tf.data.Dataset.from_tensor_slices(
(tf.random_uniform([4]),tf.random_uniform([4,10]))
)
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))

iterator = dataset3.make_initializable_iterator()
sess.run(iterator.initializer)
next1,(next2,next3) = iterator.get_next()
while True:
    try:
        print(sess.run([next1,next2,next3]))
    except tf.errors.OutOfRangeError:
        print('End of dataset')
        break

[array([0.25550473, 0.2534846 , 0.68545926, 0.38751268, 0.00655913,
       0.46028137, 0.84213626, 0.31123805, 0.30870986, 0.3978076 ],
      dtype=float32), 0.14568627, array([0.5274316 , 0.04340923, 0.08226991, 0.43311512, 0.7030449 ,
       0.00356233, 0.14944732, 0.29576075, 0.28671527, 0.35358274],
      dtype=float32)]
[array([0.43526185, 0.845196  , 0.70372593, 0.96956515, 0.96353114,
       0.23728049, 0.71388924, 0.03502381, 0.48731446, 0.43113065],
      dtype=float32), 0.089737296, array([0.92462647, 0.08505487, 0.494797  , 0.3552382 , 0.39510274,
       0.5575557 , 0.26692283, 0.51945996, 0.8677218 , 0.42040837],
      dtype=float32)]
[array([0.55766153, 0.69210577, 0.4327116 , 0.32443774, 0.18874443,
       0.10495579, 0.8226043 , 0.75262296, 0.39207542, 0.5287682 ],
      dtype=float32), 0.6567513, array([0.28131402, 0.07283938, 0.4579631 , 0.776086  , 0.83408284,
       0.90159523, 0.8683295 , 0.38935578, 0.46140444, 0.06681561],
      dtype=float32)]
[array([0.18891275,

请注意，`next1`、`next2` 和 `next3` 是由同一个操作/节点（通过 `Iterator.get_next()`创建）生成的张量。因此，评估其中任何一个张量都会使所有组件的迭代器进入下个状态。典型的迭代器消耗方会在一个表达式中包含所有组件。

## 保存迭代器状态

**`tf.contrib.data.make_saveable_from_iterator `函数通过迭代器创建一个 `SaveableObject`，该对象可用于保存和恢复迭代器（实际上是整个输入管道）的当前状态**。

以这种方式创建的可保存对象可以添加到` tf.train.Saver` 变量列表或 `tf.GraphKeys.SAVEABLE_OBJECTS` 集合中，以便采用与` tf.Variable `相同的方式进行保存和恢复。

请参阅保存和恢复，详细了解如何保存和恢复变量。

In [18]:
import tensorflow as tf

dataset = tf.data.Dataset.range(5) # 创建数据集
iterator = dataset.make_initializable_iterator() # 给这个数据集创建一个迭代器
next_element = iterator.get_next() # 返回数据集的下一条数据

# 添加运算
result = tf.add(next_element,next_element)

# 从迭代器中创建可保存的对象
saveable = tf.contrib.data.make_saveable_from_iterator(iterator)

# 通过将迭代器对象添加到可保存对象的集合中来保存迭代器的状态
tf.add_to_collection(tf.GraphKeys.SAVEABLE_OBJECTS,saveable)

# 创建保存器
saver = tf.train.Saver()

sess = tf.Session()
sess.run(iterator.initializer)
for i in range(5):
    # 如果条件满足，则保存迭代器状态
    if i > 2:
        print('保存迭代器对象...')
        saver.save(sess,save_path='./code/iterator_state')
        sess.close()
        break
    print(sess.run(result))

print()

# 重新加载迭代器对象
with tf.Session() as sess:
    print('加载迭代器...')
    saver.restore(sess,'./code/iterator_state')
    try:
        print(sess.run(result))
    except tf.errors.OutOfRangeError:
        print('End of Iterator!')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.make_saveable_from_iterator(...)`.
0
2
4
保存迭代器对象...

加载迭代器...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./code/iterator_state
6


# 读取输入数据

## 消耗NumPy数组：适合小型数据集

如果您的**所有输入数据都适合存储在内存中**，则根据输入数据创建 `Dataset` 的最简单方法是将它们转换为 `tf.Tensor `对象，并使用 `Dataset.from_tensor_slices()`。

In [19]:
# 创建numpy训练数据
import numpy as np

features = np.arange(0,24).reshape(4,6)
labels = np.array([1,0,1,0])
print('features: ',features)
print('labels: ',labels)
np.savez('./data/training_data.npz',features=features,labels=labels) # 保存

# 加载训练数据到两维numpy数组，例如np.load()
with np.load('./data/training_data.npz') as data:
    features = data['features']
    labels = data['labels']
    
# features的每一行与labels对应
assert features.shape[0] == labels.shape[0]

dataset = tf.data.Dataset.from_tensor_slices((features,labels))
iterator = dataset.make_initializable_iterator()
features,labels = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

while True:
    try:
        feature,label = sess.run([features,labels])
        print(feature,label)
    except tf.errors.OutOfRangeError:
        print('End of Dataset!')
        break

features:  [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]
labels:  [1 0 1 0]
[0 1 2 3 4 5] 1
[ 6  7  8  9 10 11] 0
[12 13 14 15 16 17] 1
[18 19 20 21 22 23] 0
End of Dataset!


请注意，上面的代码段会将 `features` 和 `labels` 数组作为` tf.constant()` 指令嵌入在 `TensorFlow` 图中。这样非常适合小型数据集，但会浪费内存，因为会多次复制数组的内容，并可能会达到` tf.GraphDef `协议缓冲区的 2GB 上限。

作为替代方案，您可以根据 `tf.placeholder()` 张量定义 `Dataset`，并在对数据集初始化 `Iterator` 时馈送 `NumPy `数组。

In [20]:
# 创建numpy训练数据
import numpy as np
import tensorflow as tf

features = np.arange(0,24).reshape(4,6)
labels = np.array([1,0,1,0])
print('feature: ',features)
print('labels: ',labels)
np.savez('./data/training_data.npz',features=features,labels=labels) # 保存

# 加载训练数据到两维numpy数组，例如np.load()
with np.load('./data/training_data.npz') as data:
    features = data['features']
    labels = data['labels']
    
# features的每一行与labels对应
assert features.shape[0] == labels.shape[0]

# 特征占位符
features_placeholder = tf.placeholder(features.dtype,features.shape) 
# 标签占位符
labels_placeholder = tf.placeholder(labels.dtype,labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder,labels_placeholder))
iterator = dataset.make_initializable_iterator()
next_features,next_labels = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})

while True:
    try:
        feature,label = sess.run([next_features,next_labels])
        print(feature,label)
    except tf.errors.OutOfRangeError:
        print('End of Dataset!')
        break

feature:  [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]
labels:  [1 0 1 0]
[0 1 2 3 4 5] 1
[ 6  7  8  9 10 11] 0
[12 13 14 15 16 17] 1
[18 19 20 21 22 23] 0
End of Dataset!


## 消耗TFRecord数据

## 消耗文本数据

很多数据集都是作为一个或多个文本文件分布的。**`tf.data.TextLineDataset `提供了一种从一个或多个文本文件中提取行的简单方法**。**给定一个或多个文件名，`TextLineDataset `会为这些文件的每行生成一个字符串值元素**。像 `TFRecordDataset `一样，`TextLineDataset` 将接受`filenames`（作为 `tf.Tensor`），因此您可以通过传递 `tf.placeholder(tf.string) `进行参数化。

`tf.data.TextLineDataset(filenames,compression_type,buffer_size)`: 由一个或多个文本文件中的行组成的数据集。
1. filenames：一个`tf.string`张量，包含一个或多个文件名

### 将两个文件上下合并

In [21]:
import tensorflow as tf

# 待读取文本数据的文件及其路径
filenames = ['./data/Article_III.txt','./data/Summary_III.txt']
# 创建数据集
# 将两个文件合并，即将后续文件内容缀到前面文件的后面
dataset = tf.data.TextLineDataset(filenames) 
iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(20):
    content = sess.run([next_content])
    print(content[0].decode('utf-8'))
    print()

article: 这是标题行。。。。。。

article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。

article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。

article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。

article: 中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询12306客服电话，就近提前办理。运费每公斤价格根据运输里程不同而不同。

article: 今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地内吸食毒品，6人全部被警方抓获，且当事人对犯案实施供认不讳。

article: 党的十八大以来，以习近平

由输出结果可知，`tf.data.TextLineDataset(filenames) `将`filenames`里的文件上下合并在一起。

### 将两个文件左右合并

In [22]:
import tensorflow as tf

# 待读取文本数据的文件及其路径
filename1 = ['./data/Article_III.txt']
filename2 = ['./data/Summary_III.txt']
# 创建数据集
# 将两个文件合并，即将后续文件内容缀到前面文件的后面
dataset1 = tf.data.TextLineDataset(filename1)
dataset2 = tf.data.TextLineDataset(filename2)
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))
iterator = dataset3.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(10):
    content = sess.run([next_content])
    article,summary = content[0][0],content[0][1]
    print(article.decode('utf-8'),'- - 分割线- - ',summary.decode('utf-8'))
    print()

article: 这是标题行。。。。。。 - - 分割线- -  summary: 这是标题行。。。。

article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人 - - 分割线- -  summary: 可穿戴技术十大设计原则

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。 - - 分割线- -  summary: 经济学人：智能手机将成为“真正的个人电脑”

article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。 - - 分割线- -  summary: 雅虎宣布剥离阿里巴巴股份

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。 - - 分割线- -  summary: 51信用卡管家，预计2015年放贷额度远超30亿

article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。 - - 分割线- -  summary: 如何选择正确的编程语言进行学习

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。 - - 分割线- -  summary: 媒体融合关键是以人为本

article: 中国

### 对文本每行进行转换、处理

默认情况下，`TextLineDataset` 会生成每个文件的每一行，这可能是不可取的（例如，如果文件以标题行开头或包含注释）。可以使用 `Dataset.skip()` 和 `Dataset.filter()` 转换来移除这些行。为了将这些转换分别应用于每个文件，我们使用 `Dataset.flat_map()` 为每个文件创建一个嵌套的` Dataset`。

In [23]:
import tensorflow as tf

# 待读取文本数据的文件及其路径
filenames = ['./data/Article_III.txt','./data/Summary_III.txt']
# 将filenames切分成两个tf.string对象,注意这里filenames仅代表上面那个列表而已
dataset = tf.data.Dataset.from_tensor_slices(filenames) 

# 用Dataset.flat_map() 用来将每个文件切分成单独的一个嵌入数据集
# 然后将其内容按顺序连接到单个 平整过的数据集
# 用 skip(1) 跳过第一行(即标题行)
dataset = dataset.flat_map(
    lambda filename:(
        tf.data.TextLineDataset(filename)
        .skip(1)
        .filter(lambda line:tf.not_equal(tf.substr(line,0,10),'#'))
    )
)

iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(18):
    content = sess.run([next_content])
    print(content[0].decode('utf-8'))
    print()

Instructions for updating:
Use `tf.strings.substr` instead of `tf.substr`.
article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。

article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。

article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。

article: 中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询12306客服电话，就近提前办理。运费每公斤价格根据运输里程不同而不同。

article: 今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地

## 消耗CSV数据

**`CSV` 文件格式是用于以纯文本格式存储表格数据的常用格式**。`tf.contrib.data.CsvDataset` 类提供了一种从符合` RFC 4180 `的一个或多个 `CSV `文件中提取记录的方法。给定一个或多个文件名以及默认值列表后，`CsvDataset` 将生成一个**元素元组**，元素类型对应于为每个 `CSV `记录提供的默认元素类型。像 `TFRecordDataset` 和 `TextLineDataset` 一样，`CsvDataset` 将接受 `filenames`（作为 `tf.Tensor`），因此您可以通过传递 `tf.placeholder(tf.string) `进行参数化。

### 创建CSV文本数据，并存储

In [24]:
import pandas as pd
import tensorflow as tf

# 创建csv格式的文本文件
with open('./data/Article_III.txt','r',encoding='utf-8') as fi:
    articles = [line.strip('\n') for line in fi.readlines()]
    
with open('./data/Summary_III.txt','r',encoding='utf-8') as fi:
    summarys = [line.strip('\n') for line in fi.readlines()]
data = pd.DataFrame({'文章':[line for line in articles],'摘要':[line for line in summarys]})
data

,文章,摘要
0,article: 这是标题行。。。。。。,summary: 这是标题行。。。。
1,article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业...,summary: 可穿戴技术十大设计原则
2,article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为...,summary: 经济学人：智能手机将成为“真正的个人电脑”
3,article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团1...,summary: 雅虎宣布剥离阿里巴巴股份
4,article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷...,summary: 51信用卡管家，预计2015年放贷额度远超30亿
5,article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行...,summary: 如何选择正确的编程语言进行学习
6,article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体...,summary: 媒体融合关键是以人为本
7,article: 中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼...,summary: 自行车不能带上火车
8,article: 今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布...,summary: 北京警方确认李代沫吸毒被捕(图)
9,article: 党的十八大以来，以习近平同志为总书记的党中央高瞻远瞩战略谋划，着力创新发展...,summary: 生态兴则文明兴生态衰则文明衰


In [25]:
data.to_csv('./data/article_summary.csv',index=False,encoding='utf-8-sig')

### 加载CSV训练数据

In [26]:
filenames = ['./data/article_summary.csv']
recored_defaults = [tf.string] *2 # 2列都是文本
dataset = tf.contrib.data.CsvDataset(filenames,recored_defaults)
iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(10):
    content = sess.run([next_content])
    print(content[0][0].decode('utf-8'),content[0][1].decode('utf-8'))
    print()

Instructions for updating:
Use `tf.data.experimental.CsvDataset(...)`.
﻿文章 摘要

article: 这是标题行。。。。。。 summary: 这是标题行。。。。

article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人 summary: 可穿戴技术十大设计原则

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。 summary: 经济学人：智能手机将成为“真正的个人电脑”

article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。 summary: 雅虎宣布剥离阿里巴巴股份

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。 summary: 51信用卡管家，预计2015年放贷额度远超30亿

article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。 summary: 如何选择正确的编程语言进行学习

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。 summary: 媒体融合关键是以人为本

article: 中国铁路总公司

#### 给CSV文件某些列提供默认值，而不是类

In [27]:
filenames = ['./data/article_summary.csv']
recored_defaults = [tf.string,[' ']] # 第一列默认是tf.string类型,第二列默认是' '
dataset = tf.contrib.data.CsvDataset(filenames,recored_defaults)
iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(10):
    content = sess.run([next_content])
    print(content[0][0].decode('utf-8'),content[0][1].decode('utf-8'))
    print()

﻿文章 摘要

article: 这是标题行。。。。。。 summary: 这是标题行。。。。

article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人 summary: 可穿戴技术十大设计原则

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。 summary: 经济学人：智能手机将成为“真正的个人电脑”

article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。 summary: 雅虎宣布剥离阿里巴巴股份

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。 summary: 51信用卡管家，预计2015年放贷额度远超30亿

article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。 summary: 如何选择正确的编程语言进行学习

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。 summary: 媒体融合关键是以人为本

article: 中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询1230

#### 读取CSV文件指定列和首行

In [28]:
filenames = ['./data/article_summary.csv']
recored_defaults = [tf.string] # 只提供选择列的数据类型或默认值即可
dataset = tf.contrib.data.CsvDataset(filenames,recored_defaults,header=True,select_cols=[0])
iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(10):
    content = sess.run([next_content])
    print(content[0][0].decode('utf-8'))
    print()

article: 这是标题行。。。。。。

article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。

article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。

article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。

article: 中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询12306客服电话，就近提前办理。运费每公斤价格根据运输里程不同而不同。

article: 今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地内吸食毒品，6人全部被警方抓获，且当事人对犯案实施供认不讳。

article: 党的十八大以来，以习近平

In [29]:
filenames = ['./data/article_summary.csv']
recored_defaults = [tf.string] # 只提供选择列的数据类型或默认值即可
dataset = tf.contrib.data.CsvDataset(filenames,recored_defaults,header=True,select_cols=[1])
iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(10):
    content = sess.run([next_content])
    print(content[0][0].decode('utf-8'))
    print()

summary: 这是标题行。。。。

summary: 可穿戴技术十大设计原则

summary: 经济学人：智能手机将成为“真正的个人电脑”

summary: 雅虎宣布剥离阿里巴巴股份

summary: 51信用卡管家，预计2015年放贷额度远超30亿

summary: 如何选择正确的编程语言进行学习

summary: 媒体融合关键是以人为本

summary: 自行车不能带上火车

summary: 北京警方确认李代沫吸毒被捕(图)

summary: 生态兴则文明兴生态衰则文明衰



# 使用Dataset.map()预处理数据

`Dataset.map(f) `转换通过将指定函数` f `应用于输入数据集的每个元素来生成新数据集。此转换基于` map() `函数（通常应用于函数式编程语言中的列表和其他结构）。函数` f` 会接受表示输入中单个元素的 `tf.Tensor` 对象，并返回表示新数据集中单个元素的 `tf.Tensor` 对象。**此函数的实现使用标准的 `TensorFlow` 指令将一个元素转换为另一个元素**。

本部分介绍了如何使用 `Dataset.map()` 的常见示例。

## 解析tf.Example协议缓冲区消息

许多输入管道都从 `TFRecord `格式的文件中提取` tf.train.Example` 协议缓冲区消息（例如这种文件使用 `tf.python_io.TFRecordWriter` 编写而成）。每个`tf.train.Example` 记录都包含一个或多个“特征”，输入管道通常会将这些特征转换为张量。

```Python
# Transforms a scalar string `example_proto` into a pair of a scalar string and
# a scalar integer, representing an image and its label, respectively.
def _parse_function(example_proto):
  features = {"image": tf.FixedLenFeature((), tf.string, default_value=""),
              "label": tf.FixedLenFeature((), tf.int64, default_value=0)}
  parsed_features = tf.parse_single_example(example_proto, features)
  return parsed_features["image"], parsed_features["label"]

# Creates a dataset that reads all of the examples from two files, and extracts
# the image and label features.
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)
```

## 解析图片数据并调整其大小

在用真实的图片数据训练神经网络时，通常需要将不同大小的图片转换为通用大小，这样就可以将它们批处理为具有固定大小的数据。

```Python
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string)
  image_resized = tf.image.resize_images(image_decoded, [28, 28])
  return image_resized, label

# A vector of filenames.
filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg", ...])

# `labels[i]` is the label for the image in `filenames[i].
labels = tf.constant([0, 37, ...])

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)
```

## 使用tf.py_func()应用任意Python逻辑

为了确保性能，我们建议您尽可能使用 `TensorFlow `指令预处理数据。不过，在解析输入数据时，调用外部 `Python `库有时很有用。为此，请在 `Dataset.map()` 转换中调用 `tf.py_func()` 指令。

```Python
import cv2

# Use a custom OpenCV function to read the image, instead of the standard
# TensorFlow `tf.read_file()` operation.
def _read_py_function(filename, label):
  image_decoded = cv2.imread(filename.decode(), cv2.IMREAD_GRAYSCALE)
  return image_decoded, label

# Use standard TensorFlow operations to resize the image to a fixed shape.
def _resize_function(image_decoded, label):
  image_decoded.set_shape([None, None, None])
  image_resized = tf.image.resize_images(image_decoded, [28, 28])
  return image_resized, label

filenames = ["/var/data/image1.jpg", "/var/data/image2.jpg", ...]
labels = [0, 37, 29, 1, ...]

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(
    lambda filename, label: tuple(tf.py_func(
        _read_py_function, [filename, label], [tf.uint8, label.dtype])))
dataset = dataset.map(_resize_function)
```

# 批处理数据集元素

![](./image/batch.jpg)

## 批处理具有相同形状的张量

最简单的批处理形式是将数据集中的 `n `个连续元素堆叠为一个元素。`Dataset.batch()` 转换正是这么做的，它与 `tf.stack()` 运算符具有相同的限制（被应用于元素的每个组件）：即对于每个组件` i`，所有元素的张量形状都必须完全相同。

In [30]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0,-100,-1)
dataset = tf.data.Dataset.zip((inc_dataset,dec_dataset))

# 批处理，每次4条数据记录
batched_dataset = dataset.batch(4)

# 构建单次迭代器
iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))
print()
print(sess.run(next_element))
print()
print(sess.run(next_element))

(array([0, 1, 2, 3], dtype=int64), array([ 0, -1, -2, -3], dtype=int64))

(array([4, 5, 6, 7], dtype=int64), array([-4, -5, -6, -7], dtype=int64))

(array([ 8,  9, 10, 11], dtype=int64), array([ -8,  -9, -10, -11], dtype=int64))


## 批处理不同维度的张量: 使用填充批处理张量

上述方法适用于具有相同大小的张量。不过，**很多模型（例如序列模型）处理的输入数据可能具有不同的大小（例如序列的长度不同）**。为了解决这种情况，可以**通过` Dataset.padded_batch()` 转换来指定一个或多个会被填充的维度，从而批处理不同形状的张量**。

`tf.data.Dataset.padded_batch(batch_size,padded_shapes,padding_values=None,drop_remainder=False)` : 将此数据集的连续元素组合到填充批中

1. batch_size：每批数据集的连续元素的个数
2. padded_shapes：tf.tensor shape或tf.int64类向量张量对象的嵌套结构，表示在批处理之前每个输入元素的各个组件应填充到的形状。任何未知维度(例如tf.TensorShape中的tf.dimension(None)或张量类对象中的-1)将被填充到每批中该维度的最大大小。
3. padding_values：填充值
4. drop_remainder：一个布尔值，表示最有一个Batch是否应该被丢弃，以防止该batch的大小小于指定的batch_size，默认是不丢弃

In [31]:
dataset = tf.data.Dataset.range(100)
# tf.fill 将输入的组件x(这里是一个值的张量)，转为tf.int32类型
# 然后用 tf.fill([x],x)来构造维度是x，并用x填充的张量
# 这个张量其实是一个一维列表，因此会随着x的增大，维度变大
dataset = dataset.map(lambda x : tf.fill([tf.cast(x,tf.int32)],x)) 

# 填充张量
# 将同一批次的张量(这里batch=4)，填充到相同的维度大小
# 这里由于padded_shape=[None]，因此会使得同一批次的张量维度相同，
# 而维度以同一批次最大的那个维度为准
dataset = dataset.padded_batch(4,padded_shapes=[None])

# 构建单次迭代器
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))
print()
print(sess.run(next_element))
print()
print(sess.run(next_element))

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]

[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]

[[ 8  8  8  8  8  8  8  8  0  0  0]
 [ 9  9  9  9  9  9  9  9  9  0  0]
 [10 10 10 10 10 10 10 10 10 10  0]
 [11 11 11 11 11 11 11 11 11 11 11]]


您可以通过 `Dataset.padded_batch() `转换为**每个组件的每个维度设置不同的填充，并且可以采用可变长度（在上面的示例中用 `None` 表示）或恒定长度**。也可以替换填充值，默认设置为` 0`

# 训练工作流程

## 处理多个周期

![](./image/repeat.jpg)

### Dataset.repeat() 设置重复多个周期

`tf.data `API 提供了两种主要方式来处理同一数据的多个周期。
要迭代数据集多个周期，最简单的方法是使用` Dataset.repeat()` 转换。例如，要创建一个将其输入重复 `10` 个周期的数据集：

In [32]:
import tensorflow as tf

# 待读取文本数据的文件及其路径
filenames = ['./data/Article_III.txt','./data/Summary_III.txt']
# 创建数据集
# 将两个文件合并，即将后续文件内容缀到前面文件的后面
dataset = tf.data.TextLineDataset(filenames)

# 创建重复2个周期的数据集，等价于将数据集复制10份
dataset = dataset.repeat(2)

# 每批次训练数据20个
dataset = dataset.batch(2)


iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

for _ in range(10):
    content = sess.run([next_content])
    print(content[0][0].decode('utf-8'),'- - - - ',content[0][1].decode('utf-8'))
    print()

article: 这是标题行。。。。。。 - - - -  article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。 - - - -  article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。 - - - -  article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。 - - - -  article: 中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询12306客服电话，就近提前办理。运费每公斤价格根据运输里程不同而不同。

article: 今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地内吸食毒品，6人全部被警方抓获，且当事人对

应用不带参数的` Dataset.repeat()` 转换将**无限次地重复输入**。`Dataset.repeat()` 转换将其参数连接起来，而不会在一个周期结束和下一个周期开始时发出信号。

### 捕获数据集耗尽时异常

如果您想在每个周期结束时收到信号，则可以编写在数据集结束时捕获 `tf.errors.OutOfRangeError `的训练循环。此时，您可以收集关于该周期的一些统计信息（例如验证错误）。

In [33]:
import tensorflow as tf

# 待读取文本数据的文件及其路径
filenames = ['./data/Article_III.txt','./data/Summary_III.txt']
# 创建数据集
# 将两个文件合并，即将后续文件内容缀到前面文件的后面
dataset = tf.data.TextLineDataset(filenames)

# 创建重复2个周期的数据集，等价于将数据集复制10份
dataset = dataset.repeat(1)

# 每批次训练数据20个
dataset = dataset.batch(2)


iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

while True:
    try:
        content = sess.run([next_content])
        print(content[0][0].decode('utf-8'),'- - - - ',content[0][1].decode('utf-8'))
        print()
    except tf.errors.OutOfRangeError:
        print('End of Dataset')
        break

article: 这是标题行。。。。。。 - - - -  article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人

article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。 - - - -  article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。

article: 2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。 - - - -  article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。

article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。 - - - -  article: 中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询12306客服电话，就近提前办理。运费每公斤价格根据运输里程不同而不同。

article: 今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地内吸食毒品，6人全部被警方抓获，且当事人对

## 随机重排输入数据

![](./image/shuffle.jpg)

`Dataset.shuffle()` 转换会使用类似于 `tf.RandomShuffleQueue` 的算法**随机重排输入数据集：它会维持一个固定大小的缓冲区，并从该缓冲区统一地随机选择下一个元素**。

`tf.data.Dataset.shuffle(buffer_size,seed=None,reshuffle_each_iteration=None)`：随机混洗数据集
1. buffer_size：tf.int64类型的标量，表示新数据集将从原数据集中采样的元素个数
2. seed：随机数种子
3. reshuffle_each_iteration：一个布尔值，如果为True，则表示每次重复数据集时，都应对其进行随机重新混合

In [34]:
import tensorflow as tf

# 待读取文本数据的文件及其路径
filenames = ['./data/Article_III.txt','./data/Summary_III.txt']
# 创建数据集
# 将两个文件合并，即将后续文件内容缀到前面文件的后面
dataset = tf.data.TextLineDataset(filenames)

# 创建重复2个周期的数据集，等价于将数据集复制10份
dataset = dataset.repeat(10)

# 混洗缓冲区中的数据
dataset = dataset.shuffle(buffer_size=10)

# 每批次训练数据20个
dataset = dataset.batch(2)


iterator = dataset.make_initializable_iterator()
next_content = iterator.get_next()
sess = tf.Session()
sess.run(iterator.initializer)

while True:
    try:
        content = sess.run([next_content])
        print(content[0][0].decode('utf-8'),'- - - - ',content[0][1].decode('utf-8'))
        print()
    except tf.errors.OutOfRangeError:
        print('End of Dataset')
        break

article: 本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人 - - - -  article: 受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。

article: 今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地内吸食毒品，6人全部被警方抓获，且当事人对犯案实施供认不讳。 - - - -  article: 2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。

article: 党的十八大以来，以习近平同志为总书记的党中央高瞻远瞩战略谋划，着力创新发展理念，大力建设生态文明，引领中华民族在伟大复兴的征途上奋勇前行。生态兴则文明兴，生态衰则文明衰——这是对人类文明发展规律的深刻总结。 - - - -  article: 雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。

article: 目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。 - - - -  article: 这是标题行。。。。。。

summary: 可穿戴技术十大设计原则 - - - -  summary: 这是标题行。。。。

summary: 雅虎宣布剥离阿里巴巴股份 - - - -  summary: 媒体融合关键是以人为本

article: 本文总结了十个

limit_output extension: Maximum message size of 10000 exceeded with 14265 characters

## 使用高阶API

`tf.train.MonitoredTrainingSession` API 简化了在分布式设置下运行 `TensorFlow `的很多方面。

`MonitoredTrainingSession `使用 `tf.errors.OutOfRangeError `表示训练已完成，因此要将其与 `tf.data `API 结合使用，我们建议使用 `Dataset.make_one_shot_iterator()`。例如：

```Python
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(32)
dataset = dataset.repeat(num_epochs)
iterator = dataset.make_one_shot_iterator()

next_example, next_label = iterator.get_next()
loss = model_function(next_example, next_label)

training_op = tf.train.AdagradOptimizer(...).minimize(loss)

with tf.train.MonitoredTrainingSession(...) as sess:
  while not sess.should_stop():
    sess.run(training_op)
```

要在 `input_fn` 中使用` Dataset`（`input_fn` 属于` tf.estimator.Estimator`），只需返回` Dataset` 即可，框架将负责为您创建和初始化迭代器。例如：

```Python
def dataset_input_fn():
  filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
  dataset = tf.data.TFRecordDataset(filenames)

  # Use `tf.parse_single_example()` to extract data from a `tf.Example`
  # protocol buffer, and perform any additional per-record preprocessing.
  def parser(record):
    keys_to_features = {
        "image_data": tf.FixedLenFeature((), tf.string, default_value=""),
        "date_time": tf.FixedLenFeature((), tf.int64, default_value=""),
        "label": tf.FixedLenFeature((), tf.int64,
                                    default_value=tf.zeros([], dtype=tf.int64)),
    }
    parsed = tf.parse_single_example(record, keys_to_features)

    # Perform additional preprocessing on the parsed data.
    image = tf.image.decode_jpeg(parsed["image_data"])
    image = tf.reshape(image, [299, 299, 1])
    label = tf.cast(parsed["label"], tf.int32)

    return {"image_data": image, "date_time": parsed["date_time"]}, label

  # Use `Dataset.map()` to build a pair of a feature dictionary and a label
  # tensor for each example.
  dataset = dataset.map(parser)
  dataset = dataset.shuffle(buffer_size=10000)
  dataset = dataset.batch(32)
  dataset = dataset.repeat(num_epochs)

  # Each element of `dataset` is tuple containing a dictionary of features
  # (in which each value is a batch of values for that feature), and a batch of
  # labels.
  return dataset
```